# Grupo A3 - Classification using feature vectors

 In this project we will extract features from the shapes present in the images in order to correctly classify the EMNIST dataset. 
 
The features to be extracted are:

TODO: choose features
- area of the black pixels (foreground/shape)
- area of the white pixels (background)
- perimeter of the shape
- bounding-box area
- major axis length
- solidity
- extent
- equivalent-diameter
- centroid (x and y coordinates)
- convex_area

The classifiers that will be used are: 
- Nearest Neighbors
- SVC (SVM for classification, C-Support Vector Classification.)
- Gaussian Process
- Decision Tree
- Random Forest
- MLP (Multi-Layer Perceptron)
- Ada Boost
- Gaussian NB
- Quadratic Discriminant Analysis

We will use the classifiers from scikit-learn and follow the scheme present in 

#  Loading Data

## Imports

In [64]:
import sys
sys.path.append('../../')

from sarpy.datasets import load_emnist
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

print("teste")

teste


## Loading data set and binarizing the images

In [222]:
X_train, y_train, X_test, y_test, X_valid, y_valid, mapping, nb_classes = load_emnist('balanced', validation=True)

X_train = X_train > 0
X_train = X_train.astype(int)
X_train = np.squeeze(X_train)

X_test = X_test > 0
X_test = X_test.astype(int)
X_test = np.squeeze(X_test)

X_valid = X_valid > 0
X_valid = X_valid.astype(int)
X_valid = np.squeeze(X_valid)

reshape testing: 100%|██████████| 18800/18800 [00:00<00:00, 39842.59it/s]


Train size: 94000
Test size: 18800
Validation size: 18800
# classes: 47


# Extracting Features

## Imports

In [164]:
from skimage.measure import label, regionprops

## Feature extraction

In [262]:
def feature_extraction(image):
    features = [ [] for _ in range(11)]
    label_img = label(image)
    props = regionprops(label_img)
    for j in range(0, len(props)):
        features[0].append(props[j].area)
        features[1].append(props[j].perimeter)
        features[2].append(props[j].bbox_area)
        features[3].append(props[j].major_axis_length)
        features[4].append(props[j].solidity)
        features[5].append(props[j].extent)
        features[6].append(props[j].equivalent_diameter)
        features[7].append(props[j].centroid[0])
        features[8].append(props[j].centroid[1])
        features[9].append(props[j].convex_area)
        features[10].append(props[j].filled_area)
    return features


In [267]:
f = []
#isso está demorando bastante 
for i in range(0, len(X_train)):
    f.append(feature_extraction(X_train[i]))

for i in range(0, len(f[19])):
    for j in range(0, len(f[19][i])):
        print(f[19][i][j])
    print("\n")

# print(f)

/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION

138
7
8


89.42640687119285
6.82842712474619
7.414213562373095


336
9
9


25.075522444278086
3.0237157840738176
3.4641016151377544


0.6509433962264151
1.0
1.0


0.4107142857142857
0.7777777777777778
0.8888888888888888


13.255453865237978
2.985410660720923
3.1915382432114616


13.021739130434783
6.0
15.125


18.905797101449274
3.7142857142857144
1.875


212
7
8


167
7
8




# Fitting Classifiers

# Results